# One-class SVM 動作確認（再試行）

scikit-learn の <b>One-class SVM</b> のテストを再試行しました。

## (0) 結果と結論

結果としては、

- 非線形カーネルでは、feature 抽出件数が０件（質問文に、学習時のボキャブラリが出現しない）時、アノマリー検出されない

 非線形カーネル以外では、全てのケースで feature０件時はアノマリーとして検出されるのを確認しております。
 
 
- 「する」というfeatureが抽出された質問文は、feature抽出件数＜５件のケースで、アノマリーと検出されない

 今回の検証ではデータを大幅に（７倍近くに）増やしたのですが、この点は改善することはありませんでした。
 
 ただし、feature抽出件数が５件以上になると「する」の存在にかかわらず、アノマリーと検出されるようです（個々のfeatureの影響度合が弱くなるせいか？）。

~~~
feature 抽出結果の例：
'要素技術は自然languageの機械learningですか？'-->[]
'人生相談をしたいのですが？'-->[する=1.000]
'難解なプログラミング技術を調達？'-->[調達=1.000]
'何か習い事をしますか？'-->[する=0.707 何=0.707]
'何か習い事がいいですか？'-->[いい=0.707 何=0.707]
'何か習い事をしたほうがいいですか？'-->[いい=0.577 する=0.577 何=0.577]
'何か習い事を行うほうがいいですか？'-->[いい=0.577 何=0.577 行う=0.577]
'会社を辞めたいのですが誰に相談するのがいいですか？'-->[いい=0.500 する=0.500 会社=0.500 誰=0.500]
'会社を辞めたいのですが誰に相談を行うのがいいですか？'-->[いい=0.500 会社=0.500 行う=0.500 誰=0.500]
'有給休暇を取って海外に行き旅行する意向があります。'-->[する=0.447 休暇=0.447 取る=0.447 海外=0.447 行く=0.447]
'有給休暇を取って海外に行きたいと思っています。'-->[休暇=0.447 取る=0.447 思う=0.447 海外=0.447 行く=0.447]
~~~

結論としては、

- One-class SVMのカーネルを非線形カーネルとする

 
- 「する」などの高頻出 feature の影響度合いを、カスタムカーネル（またはTF-IDFの見直し）などにより弱める
 
といったところで上記事象は回避できるのかと思われます。

## (1) テストデータ／環境準備

マイオペで使用しているテストデータ（learning/tests/engine/fixtures/ 配下のCSVファイル）をベースに動作確認を行います。

動作確認にあたっては、MySQLdb に接続できないため、ローカル環境テスト用の Bot クラスを使用しています。

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

from prototype.modules import TestTool

In [2]:
'''
    データファイルは、既存の訓練データを別場所にコピーしてから使用します
    テストデータは、csv_file_name で指定した複数件のファイルを使用します。
'''
csv_file_names = [
    'test_benefitone_conversation.csv',
    'test_daikin_conversation.csv',
    'test_ptna_conversation.csv',
    'test_septeni_conversation.csv'
]
copied_csv_file_paths = TestTool.copy_testdata_csv(learning_dir, csv_file_names)
copied_csv_file_paths

CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_benefitone_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_daikin_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_ptna_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_septeni_conversation.csv]


['/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_benefitone_conversation.csv',
 '/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_daikin_conversation.csv',
 '/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_ptna_conversation.csv',
 '/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_septeni_conversation.csv']

## (2) TF-IDFベクターの準備

Bot クラス内に組み込まれている __build_training_set_from_csv 関数をバラして実行しています。

In [3]:
'''
    初期設定
    データファイル、エンコードを指定
    内容は、learn.py を参考にしました。    
'''
from learning.core.learn.learning_parameter import LearningParameter
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': None,
    # 'algorithm': LearningParameter.ALGORITHM_NAIVE_BAYES
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    # 'params_for_algorithm': { 'C': 200 }
    'params_for_algorithm': {}
}
learning_parameter = LearningParameter(attr)

bot_id = 7777
csv_file_encoding = 'utf-8'

### (2-1) 訓練データのTF-IDFベクター

In [4]:
'''
    訓練データの生成（内部で TF-IDF 処理を実行）

    TrainingMessageFromCsv は、
    ファイル名のリスト（copied_csv_file_paths）を受け取ると、
    Dataset を連結してくれるようカスタマイズしてあります。
'''
#from learning.core.training_set.training_message_from_csv import TrainingMessageFromCsv
from prototype.modules.training_message_from_csv import TrainingMessageFromCsv
training_set = TrainingMessageFromCsv(bot_id, copied_csv_file_paths, learning_parameter, encoding=csv_file_encoding)
build_training_set_from_csv = training_set.build()

X = build_training_set_from_csv.x
y = build_training_set_from_csv.y

2017/03/21 PM 10:32:32 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 28272
2017/03/21 PM 10:32:32 TextArray#__init__ start
2017/03/21 PM 10:32:48 TextArray#to_vec start
2017/03/21 PM 10:32:48 TextArray#to_vec end


### (2-2) 外れデータのTF-IDFベクター

In [5]:
'''
    マイオペのプロダクション・コードと同じように、
    訓練データ作成時と同じベクトライザーを使用します。
'''
from learning.core.training_set.text_array import TextArray

test_X = [
    '要素技術は自然languageの機械learningですか？', # まったくfeatureが抽出されない質問文

    '人生相談をしたいのですが？', # featureが１件抽出されるな質問文「する」を含む
    '難解なプログラミング技術を調達？', # featureが１件抽出される質問文「する」を含まない

    '何か習い事をしますか？', # featureが２件抽出される質問文「する」を含む
    '何か習い事がいいですか？', # featureが２件抽出される質問文「する」を含まない
    
    '何か習い事をしたほうがいいですか？', # featureが３件抽出される質問文「する」を含む
    '何か習い事を行うほうがいいですか？', # featureが３件抽出される質問文「する」を含まない
    
    '会社を辞めたいのですが誰に相談するのがいいですか？', # featureが４件抽出される質問文「する」を含む
    '会社を辞めたいのですが誰に相談を行うのがいいですか？', # featureが４件抽出される質問文「する」を含まない

    '有給休暇を取って海外に行き旅行する意向があります。', # featureが５件抽出される質問文「する」を含む
    '有給休暇を取って海外に行きたいと思っています。', # featureが５件抽出される質問文「する」を含まない
]
vectorizer = training_set.body_array.vectorizer
text_array = TextArray(test_X, vectorizer=vectorizer)

'''
    外れデータのTF-IDFベクターを取得
'''
X_error = text_array.to_vec()

2017/03/21 PM 10:32:48 TextArray#__init__ start
2017/03/21 PM 10:32:48 TextArray#to_vec start
2017/03/21 PM 10:32:48 TextArray#to_vec end


### (2-3) 外れデータのfeatureを確認

In [6]:
'''
    feature をダンプするためのツール
'''
def get_item_from_vocabulary(vocabulary, index):
    for k, v in vocabulary.items():
        if v == index:
            return k

    return None

def dump_features(arr, vocabulary):
    features_str = ''

    for i, v in enumerate(arr):
        if v == 0.0:
            continue

        if features_str != '':
            features_str += ' '
        
        item = get_item_from_vocabulary(vocabulary, i)
        features_str += '%s=%0.3f' % (item, v)

    return '[' + features_str + ']'

def get_dumped_features(X_error, vocabulary):
    dumped_features = []

    for i, label in enumerate(X_error):
        arr = X_error[i].toarray()[0]
        dump_str = dump_features(arr, vocabulary)
        dumped_features.append('index=%d%s' % (i, dump_str))

    return dumped_features

In [7]:
vocabulary = text_array._vectorizer.vocabulary_
dumped_features = get_dumped_features(X_error, vocabulary)
for d in dumped_features:
    print(d)

index=0[]
index=1[する=1.000]
index=2[調達=1.000]
index=3[する=0.707 何=0.707]
index=4[いい=0.707 何=0.707]
index=5[いい=0.577 する=0.577 何=0.577]
index=6[いい=0.577 何=0.577 行う=0.577]
index=7[いい=0.500 する=0.500 会社=0.500 誰=0.500]
index=8[いい=0.500 会社=0.500 行う=0.500 誰=0.500]
index=9[する=0.447 休暇=0.447 取る=0.447 海外=0.447 行く=0.447]
index=10[休暇=0.447 取る=0.447 思う=0.447 海外=0.447 行く=0.447]


## (3) One-class SVMで学習（複数の手順を使用して比較）

### (3-1) 非線形カーネルを使用（RBF）

In [8]:
'''
    One-class SVM with non-linear kernel, anomary is half
        fn(i,j) = exp(-gamma*|i-j|^2)
'''
from sklearn import svm
clf_rbf = svm.OneClassSVM(
    kernel='rbf', # RBFカーネルを使用
    gamma=0.1,    # 係数（デフォルト＝[1/feature数]）
    nu=0.5        # 外れ判定される件数の見積もりを全体の5割と想定
    ) 
clf_rbf.fit(X)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='rbf',
      max_iter=-1, nu=0.5, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

### (3-2) 多項式カーネルを使用

In [9]:
'''
    One-class SVM with Polynomial kernel, anomary is half
        fn(i,j) = (gamma*i'*j + coef0)^degree
'''
from sklearn import svm
clf_poly = svm.OneClassSVM(
    kernel='poly', # Polynomialカーネルを使用
    gamma=0.1,     # 係数 （デフォルト＝[1/feature数]）
    coef0=1.0,     # オフセット（デフォルト＝0）
    degree=2,      # 乗数（デフォルト＝3）
    nu=0.5         # 外れ判定される件数の見積もりを全体の5割と想定
    ) 
clf_poly.fit(X)

OneClassSVM(cache_size=200, coef0=1.0, degree=2, gamma=0.1, kernel='poly',
      max_iter=-1, nu=0.5, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

### (3-3) シグモイドカーネルを使用

In [10]:
'''
    One-class SVM with Sigmoid kernel, anomary is half
        fn(i,j) = tanh(gamma*i'*j + coef0)
'''
from sklearn import svm
clf_sigmoid = svm.OneClassSVM(
    kernel='sigmoid', # シグモイドカーネルを使用
    gamma=0.1,        # 係数（デフォルト＝[1/feature数]）
    coef0=0.5,        # オフセット（デフォルト＝0）
    nu=0.5            # 外れ判定される件数の見積もりを全体の5割と想定
    ) 
clf_sigmoid.fit(X)

OneClassSVM(cache_size=200, coef0=0.5, degree=3, gamma=0.1, kernel='sigmoid',
      max_iter=-1, nu=0.5, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

### (3-4) 線形カーネルを使用

In [11]:
'''
    One-class SVM with linear kernel, anomary is half
        fn(i,j) = i'*j
'''
from sklearn import svm
clf_linear = svm.OneClassSVM(
    kernel='linear', # linearカーネルを使用
    nu=0.5           # 外れ判定される件数の見積もりを全体の5割と想定
    ) 
clf_linear.fit(X)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto',
      kernel='linear', max_iter=-1, nu=0.5, random_state=None,
      shrinking=True, tol=0.001, verbose=False)

### (3-5) ご参考：線形カーネルベースのカスタムカーネルを使用

詳しくは<a href="11.ipynb"><b>こちらをご参照</b></a>

In [12]:
'''
    カスタムカーネル
'''
from scipy.sparse import csr_matrix
import numpy as np

def clear_features(matrix):
    n_sample = matrix.shape[0]
    n_feature = matrix.shape[1]
    
    for i in range(0, n_sample):
        row = matrix.getrow(i)
        cnt_feature = row.getnnz()
        if cnt_feature > 1:
            continue # feature が２件以上あるサンプルはパス

        # feature が１件以下の場合は、全列をゼロクリア
        matrix[i] = csr_matrix((1, n_feature))

def svm_custom_kernel(X, Y):
    # feature数が２件未満のサンプルは、feature が無いものと扱い、
    # 一律アノマリーと判定させます <---これはひとつの例です
    clear_features(X)
    
    # 線形カーネルと同様に演算します
    return np.dot(X, Y.T)

In [13]:
'''
    One-class SVM with customized kernel, anomary is checked in kernel
        fn(i,j) = i'*j
'''
from sklearn import svm
clf_custom = svm.OneClassSVM(
    kernel=svm_custom_kernel # カスタムカーネルを使用
    ) 
clf_custom.fit(X)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto',
      kernel=<function svm_custom_kernel at 0x10b44c9d8>, max_iter=-1,
      nu=0.5, random_state=None, shrinking=True, tol=0.001, verbose=False)

## (4) 外れデータを使って予測

カーネルを変更すると、予測結果が若干変わってきます。

In [14]:
'''
    外れデータを使用して、外れ判定を実行してみます
'''
y_pred_by_rbf = clf_rbf.predict(X_error)
y_pred_by_poly = clf_poly.predict(X_error)
y_pred_by_sigmoid = clf_sigmoid.predict(X_error)
y_pred_by_linear = clf_linear.predict(X_error)
y_pred_by_custom = clf_custom.predict(X_error)

#### X_error の 質問文と feature はこちら（再掲）

In [15]:
for d in dumped_features:
    print(d)

index=0[]
index=1[する=1.000]
index=2[調達=1.000]
index=3[する=0.707 何=0.707]
index=4[いい=0.707 何=0.707]
index=5[いい=0.577 する=0.577 何=0.577]
index=6[いい=0.577 何=0.577 行う=0.577]
index=7[いい=0.500 する=0.500 会社=0.500 誰=0.500]
index=8[いい=0.500 会社=0.500 行う=0.500 誰=0.500]
index=9[する=0.447 休暇=0.447 取る=0.447 海外=0.447 行く=0.447]
index=10[休暇=0.447 取る=0.447 思う=0.447 海外=0.447 行く=0.447]


#### 非線形カーネルの場合

明らかに異常な質問文（index0＝feature抽出件数０の質問文）に対して、正常と判定されてしまいます。
 
これはプロダクションへの選択肢としては無しかと存じます。
 
また、「する」というfeatureが抽出された質問文（index1, 3, 5, 7）は、アノマリーと判定されませんでした。

In [16]:
y_pred_by_rbf

array([ 1.,  1., -1.,  1., -1.,  1., -1.,  1., -1., -1., -1.])

#### 多項式カーネル、シグモイドカーネル、線形カーネルの場合

結果は全て同じですが、明らかに異常な質問文（index0）は、アノマリーと判定されました。

ただし「する」というfeatureが抽出された質問文（index1, 3, 5, 7）は、アノマリーと判定されませんでした。

In [17]:
y_pred_by_poly

array([-1.,  1., -1.,  1., -1.,  1., -1.,  1., -1., -1., -1.])

In [18]:
y_pred_by_sigmoid

array([-1.,  1., -1.,  1., -1.,  1., -1.,  1., -1., -1., -1.])

In [19]:
y_pred_by_linear

array([-1.,  1., -1.,  1., -1.,  1., -1.,  1., -1., -1., -1.])

#### ご参考：カスタムカーネルの場合

明らかに異常な質問文（index0）は、アノマリーと判定されました。

同時に、１件しかfeatureが抽出されない質問文（index1, 2）も、アノマリーと判定されます（これはカスタムカーネルに起因する挙動）。

あとはベースの線形カーネルと同じ結果になります。

In [20]:
y_pred_by_custom

array([-1., -1., -1.,  1., -1.,  1., -1.,  1., -1., -1., -1.])